<a href="https://colab.research.google.com/github/CoderAnubhav/Gideon-AI-Chatbot/blob/master/Gideon_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:

class Gideon:

          def __init__(self):
                import subprocess
                #subprocess.call(['pip', 'install', 'tensorflow-text'])
                subprocess.call(['pip', 'install', 'wikipedia'])
               

          def load_data(self):
                #import subprocess
                #subprocess.call(['pip', 'install', 'tensorflow-text'])
                from urllib.request import urlopen
                import pandas as pd
                import tensorflow_hub as hub
                #import tensorflow_text
                import joblib
                import requests

                

                pd.set_option('display.max_rows', None)
                pd.set_option('display.max_columns', None)                                                                     # for  removing all form of output truncations
                pd.set_option('display.width', None)
                pd.set_option('display.max_colwidth',None)
                                                                                                                                              
                url='https://raw.githubusercontent.com/CoderAnubhav/Gideon-AI-Chatbot/master/WHO_FAQ1.xlsx'                    #link for the dataset        
                response_encodings_url='https://raw.githubusercontent.com/CoderAnubhav/Gideon-AI-Chatbot/master/enc_data.pkl'  #link for the response encodings on the dataset given by the above url using Universal Sentence Encoder multilingual Qa 
                self.data=pd.read_excel(url,index_col=False)
                self.data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
                      'Unnamed: 5', 'Unnamed: 6','Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
                      'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
                      'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
                      'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
                      'Unnamed: 22', 'Unnamed: 23'],axis=1,inplace=True)
                      
                self.module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')
                self.response_encodings=joblib.load(urlopen(response_encodings_url))                                           # Load module containing USE

            
          def preprocess_sentences(self,input_sentences):
              import tensorflow as tf
              import numpy as np
              import pandas as pd
              import re

              return [re.sub(r'(covid-19|covid)', 'coronavirus', input_sentence, flags=re.I) 
                  for input_sentence in input_sentences]

          def generateResponseUtil(self,input_ques):
              import tensorflow as tf
              import numpy as np
              import pandas as pd

                #response_encodings=joblib.load(urlopen(response_encodings_url))                                 

              question_encodings = self.module.signatures['question_encoder'](
                  tf.constant(self.preprocess_sentences(input_ques))
              )['outputs']
        
              # Get the responses
              test_responses = self.data.Answer[np.argmax(np.inner(question_encodings, self.response_encodings), axis=1)]
        
              # Show them in a dataframe
              df=pd.DataFrame({'Test Questions': input_ques, 'Test Responses': test_responses})
              return df['Test Responses'].to_string(index=False)


          def welcome(self,user_response):                                                                    #Welcome Response
              import random
                                                                                  
              self.welcome_input = ("hello gideon","hi gideon","hey gideon")
              self.welcome_response = ["hi", "hey", "*nods*", "hi there", "hello"]
              if user_response.lower() in self.welcome_input:
                    return random.choice(self.welcome_response)

          def generateResponse(self,user_ques):
              robo_response=''
              if user_ques != "" :
                user_ques=[user_ques]
                user_ques=self.preprocess_sentences(user_ques)
                robo_response+=self.generateResponseUtil(user_ques)
            
              print("Gideon : "+robo_response)

           
          def wikipedia_query(self,input):                                                                #Generate Wikipedia Response
              import wikipedia as wk
              import re
                                                                                      
              reg_ex = re.search('wiki (.*)', input)
              try:
                  if reg_ex:
                      topic = reg_ex.group(1)
                      wiki = wk.summary(topic, sentences = 3)
                      return wiki
              except Exception as e:
                      return 0

          def google_query(self,query,index=0):
              from bs4 import BeautifulSoup
              from googlesearch import search
              from lxml import  html
              import string
              import requests
              
              fallback="Gideon : Sorry,I cannot find a response in google"
              reg_ex = re.search('google (.*)', query)
              query=reg_ex.group(1)
          
              try:
                search_result_list=list(search(query,tld="co.in",num=10,stop=3,pause=1))
                page=requests.get(search_result_list[index])
                tree=html.fromstring(page.content)
                soup=BeautifulSoup(page.content,features="lxml")
                article_text=''
                article=soup.findAll('p')
                for element in article:
                  article_text+='\n'+''.join(element.findAll(text=True))
                print("Gideon : Showing response from Google...")
                print(article_text)
          
          
              except:
                print(fallback)

          def chatbot_query(self):
              flag=True
              print("***************************************************")
              print("My name is Gideon and I'm a COVID-19 assistance chatbot to resolve your general queries about COVID-19 in multiple languages.")
              print("I sometime give illogical answers. If you are unsatisfied with my response you can ask me to get responses from google or wikipedia by typing google (or wiki) followed by your query")
              print("You can start a conversation with me by typing Hello Gideon or Hi Gideon")
              print("If you want to exit,type bye")
              while(flag==True):
                  user_response = input()
                  user_response=user_response.lower()
                  if (user_response not in ['bye','shutdown','exit', 'quit','bye gideon']):
                      if (user_response=='thanks' or user_response=='thank you' or user_response=='okay' or user_response=='ok'):
                          print("Gideon : You are welcome..")
                          print("Gideon : Do you want to continue?(Yes/No)")
                          flag=True if input().lower()=='yes' else False
          
                      elif "wiki" in user_response:
                          w=self.wikipedia_query(user_response)
                          print("Gideon : Showing data from Wikipedia..")
                          if w==0:
                            print("No data avilable on wikipedia for the following query")
                          else:
                            print(w)
          
                      elif "google" in user_response:
                          self.google_query(user_response)
          
                      else:
                          if(self.welcome(user_response)!=None):
                              print("Gideon : "+self.welcome(user_response))
                          else:
                              self.generateResponse(user_response)
          
                  else:
                    flag=False
          
              if flag==False:
                print("Gideon : It's a pleasure talking to you.Stay Safe.Bye!!! ")

                  

In [ ]:
gid=Gideon()
gid.load_data()
gid.chatbot_query()